In [1]:
import os
import torch
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision import transforms as T
from PIL import Image
import cv2
import numpy as np

In [2]:
def load_trained_model(model_path, num_classes, device):
    model = maskrcnn_resnet50_fpn(pretrained=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                        hidden_layer,
                                                        num_classes)

    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

In [3]:
def apply_mask(model, input_dir, output_dir, device, score_threshold=0.7):
    """
    Áp dụng mô hình Mask R-CNN để tạo mask cho các ảnh trong thư mục.
    Gộp tất cả mask của các đối tượng trong một ảnh vào một file duy nhất.

    Args:
        model: Mô hình Mask R-CNN.
        input_dir: Thư mục chứa ảnh đầu vào.
        output_dir: Thư mục lưu mask đầu ra.
        device: Thiết bị ('cuda' hoặc 'cpu').
        score_threshold: Ngưỡng điểm tin cậy.
    """
    model.eval()
    os.makedirs(output_dir, exist_ok=True)
    transform = T.Compose([T.ToTensor()])

    for img_name in os.listdir(input_dir):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            img_path = os.path.join(input_dir, img_name)
            try:
                img = Image.open(img_path).convert("RGB")
            except (FileNotFoundError, IOError) as e:
                print(f"Lỗi: Không thể mở ảnh {img_name}. Lý do: {e}. Bỏ qua.")
                continue

            img_tensor = transform(img).unsqueeze(0).to(device)

            with torch.no_grad():
                predictions = model(img_tensor)

            # 1. Tạo mask trống
            combined_mask = np.zeros((img.size[1], img.size[0]), dtype=np.uint8)  # H x W

            # 2. Lặp qua các detections và "vẽ" mask lên mask trống
            for i in range(len(predictions[0]['masks'])):
                score = predictions[0]['scores'][i].item()
                if score > score_threshold:
                    mask = predictions[0]['masks'][i, 0].cpu().numpy()
                    mask = (mask > 0.5).astype(np.uint8) * 255  # Mask nhị phân (0 và 255)

                    # "Vẽ" mask con lên mask tổng hợp
                    combined_mask = np.maximum(combined_mask, mask) # Hoặc: combined_mask[mask > 0] = 255


            # 3. Lưu mask tổng hợp
            # --- Thay đổi ở đây ---
            output_path = os.path.join(output_dir, img_name)  # Sử dụng trực tiếp img_name
            # Đảm bảo phần mở rộng là .png (nếu cần)
            base, ext = os.path.splitext(output_path)
            if ext.lower() != '.jpg':
                output_path = base + '.jpg'

            try:
                cv2.imwrite(output_path, combined_mask)
            except Exception as e:
                print(f"Lỗi lưu mask: {e}")

            print(f"Đã xử lý mask cho {img_name}")

In [4]:
try:
    torch.ops.torchvision.nms(torch.rand(10, 4, device='cuda'), torch.rand(10, device='cuda'), 0.5)
    device = torch.device('cuda')
    print("CUDA NMS is available. Using GPU.")
except NotImplementedError:
    device = torch.device('cpu')
    print("CUDA NMS is NOT available. Using CPU (SLOW).  Consider reinstalling PyTorch/Torchvision with CUDA support.")
except RuntimeError as e: #cacth RuntimeError
    if "CUDA driver" in str(e):
        device = torch.device('cpu')
        print("CUDA driver is outdated. Please update to lateset driver")
    else:
        device = torch.device('cpu')
        print("CUDA NMS is NOT available. Using CPU (SLOW). Consider reinstalling PyTorch/Torchvision with CUDA support.")


CUDA NMS is available. Using GPU.


In [5]:
os.mkdir('/kaggle/working/train_masks')
os.mkdir('/kaggle/working/val_masks')

In [6]:
num_classes = 2  # Thay đổi số này nếu bạn huấn luyện với số lớp khác
model_path = '/kaggle/input/rcnn-re-train/pytorch/default/1/rcnn_re_train.pth'  # Thay đổi đường dẫn đến file .pth của bạn!
input_image_dir = '/kaggle/input/datatset-mask/ChessPieces_Dataset_Mask/train_images'   # Thay đổi đường dẫn đến thư mục chứa ảnh cần tạo mask
output_mask_dir = '/kaggle/working/train_masks'  

In [7]:
model = load_trained_model(model_path, num_classes, device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:04<00:00, 25.1MB/s]
<ipython-input-2-5f25ddbaf20d>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.co

In [17]:
apply_mask(model, input_image_dir, output_mask_dir, device, score_threshold=0.5)

Đã xử lý mask cho original_normalized_bishop_283.jpg
Đã xử lý mask cho original_normalized_bishop_391.jpg
Đã xử lý mask cho contrast_normalized_pawn_597.jpg
Đã xử lý mask cho brightness_normalized_knight_224.jpg
Đã xử lý mask cho contrast_normalized_pawn_251.jpg
Đã xử lý mask cho brightness_normalized_king_257.jpg
Đã xử lý mask cho rotate_45_normalized_knight_207.jpg
Đã xử lý mask cho rotate_-45_normalized_rook_568.jpg
Đã xử lý mask cho flipped_x_normalized_bishop_59.jpg
Đã xử lý mask cho contrast_normalized_rook_306.jpg
Đã xử lý mask cho rotate_45_normalized_knight_363.jpg
Đã xử lý mask cho rotate_45_normalized_knight_66.jpg
Đã xử lý mask cho original_normalized_king_484.jpg
Đã xử lý mask cho rotate_-45_normalized_queen_128.jpg
Đã xử lý mask cho original_normalized_rook_533.jpg
Đã xử lý mask cho original_normalized_pawn_166.jpg
Đã xử lý mask cho brightness_normalized_pawn_313.jpg
Đã xử lý mask cho flipped_x_normalized_queen_14.jpg
Đã xử lý mask cho original_normalized_rook_460.jpg
Đã 

In [ ]:
num_classes = 2  # Thay đổi số này nếu bạn huấn luyện với số lớp khác
model_path = '/kaggle/input/rcnn-re-train/pytorch/default/1/rcnn_re_train.pth'  # Thay đổi đường dẫn đến file .pth của bạn!
input_image_dir = '/kaggle/input/datatset-mask/ChessPieces_Dataset_Mask/val_images'   # Thay đổi đường dẫn đến thư mục chứa ảnh cần tạo mask
output_mask_dir = '/kaggle/working/val_masks' 

In [ ]:
model = load_trained_model(model_path, num_classes, device)

<ipython-input-2-5f25ddbaf20d>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


In [ ]:
apply_mask(model, input_image_dir, output_mask_dir, device, score_threshold=0.5)

Đã xử lý mask cho normalized_bishop_140.jpg
Đã xử lý mask cho normalized_king_132.jpg
Đã xử lý mask cho normalized_queen_440.jpg
Đã xử lý mask cho normalized_pawn_531.jpg
Đã xử lý mask cho normalized_queen_370.jpg
Đã xử lý mask cho normalized_king_433.jpg
Đã xử lý mask cho normalized_king_207.jpg
Đã xử lý mask cho normalized_rook_238.jpg
Đã xử lý mask cho normalized_queen_289.jpg
Đã xử lý mask cho normalized_king_339.jpg
Đã xử lý mask cho normalized_pawn_588.jpg
Đã xử lý mask cho normalized_pawn_10.jpg
Đã xử lý mask cho normalized_queen_196.jpg
Đã xử lý mask cho normalized_knight_54.jpg
Đã xử lý mask cho normalized_bishop_556.jpg
Đã xử lý mask cho normalized_knight_439.jpg
Đã xử lý mask cho normalized_pawn_167.jpg
Đã xử lý mask cho normalized_knight_591.jpg
Đã xử lý mask cho normalized_king_206.jpg
Đã xử lý mask cho normalized_pawn_73.jpg
Đã xử lý mask cho normalized_king_110.jpg
Đã xử lý mask cho normalized_rook_490.jpg
Đã xử lý mask cho normalized_queen_446.jpg
Đã xử lý mask cho norm

In [21]:
!zip -r /kaggle/working/masks.zip /kaggle/working/train_masks /kaggle/working/val_masks

  adding: kaggle/working/train_masks/ (stored 0%)
  adding: kaggle/working/train_masks/brightness_normalized_bishop_45.jpg (deflated 9%)
  adding: kaggle/working/train_masks/brightness_normalized_pawn_393.jpg (deflated 13%)
  adding: kaggle/working/train_masks/rotate_45_normalized_queen_302.jpg (deflated 10%)
  adding: kaggle/working/train_masks/contrast_normalized_bishop_21.jpg (deflated 9%)
  adding: kaggle/working/train_masks/brightness_normalized_bishop_125.jpg (deflated 12%)
  adding: kaggle/working/train_masks/contrast_normalized_king_426.jpg (deflated 17%)
  adding: kaggle/working/train_masks/rotate_45_normalized_knight_164.jpg (deflated 9%)
  adding: kaggle/working/train_masks/flipped_x_normalized_queen_530.jpg (deflated 10%)
  adding: kaggle/working/train_masks/flipped_x_normalized_rook_69.jpg (deflated 7%)
  adding: kaggle/working/train_masks/rotate_-45_normalized_king_385.jpg (deflated 7%)
  adding: kaggle/working/train_masks/rotate_45_normalized_bishop_485.jpg (deflated 10%